In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from collections import deque
from threading import Thread
import os
import logging
import time
import asyncio
import random
import wandb

if "freqtrade" not in os.listdir():
    import nest_asyncio
    os.chdir("..")
    nest_asyncio.apply()
    # Uncomment if you want to enable Freqtrade Logging
#     logging.basicConfig(format='%(asctime)-15s - %(message)s')
#     logging.root.setLevel(logging.WARNING)

from freqtrade.commands.data_commands import start_download_data
PATH_MOUNT = Path.cwd().parent / "mount"
PATH_LOCAL_DATADIR = PATH_MOUNT / "data"

In [2]:
class Task:
    
    def __init__(self, pair: str, timeframe: str, num_days: int):
        self.pair = [pair]
        self.timeframe = [timeframe]
        self.num_days = num_days

        
class Downloader:
    
    def __init__(self, task_list: list, exchange: str, path_datadir: Path):
        self.task_queue = deque(task_list)
        self.print_queue = deque()
        self.exchange = exchange
        self.path_datadir = path_datadir / exchange
        print(f"Got {len(self.task_queue)} tasks.")
        print(f"Datadir: {self.path_datadir}")
        
    def start(self, num_threads: int):
        print("Starting threads")
        threads = []
        
        for i in range(num_threads):
            thread = Thread(target=self._download_job, args=(i,), daemon=True)
            thread.start()
            threads.append(thread)
        
        print_thread = Thread(target=self._print_job, daemon=True)
        print_thread.start()
        threads.append(print_thread)
    
        [it.join() for it in threads]
    
    def _download_job(self, thread_num: int):
        time.sleep(thread_num * 3)
        self._print(f"Starting worker #{thread_num}")
        asyncio.set_event_loop(asyncio.new_event_loop())
        
        while len(self.task_queue) > 0:
            try:
                task = self.task_queue.pop()
            except Exception as e:
                self._print(f"LOOP ERROR at Thread #{thread_num}: {e}.")
                continue
            args = {
                "timeframes": task.timeframe,
                "pairs": task.pair,
                "exchange": self.exchange,
                "days": task.num_days,
                "datadir": self.path_datadir,
                "verbosity": logging.ERROR,
            }
            self._print(
                f"Thread #{thread_num} - Download {task.pair} {task.timeframe}. Tasks left: {len(self.task_queue)}"
            )
            done = False
            while not done:
                # Fault Tolerance
                try:
                    start_download_data(args)
                    done = True
                except Exception as e:
                    self._print(f"DOWNLOAD ERROR at Thread #{thread_num}: {e}. ({task.pair} {task.timeframe})")
                    pass
            
            time.sleep(random.randint(1, 10))
    
    def _print(self, text: str):
        self.print_queue.append(text)

    def _print_job(self):
        while len(self.task_queue) > 0:
            while len(self.print_queue) > 0:
                print(self.print_queue.pop())
                
    def _optimize_num_days(task: Task):
        """
        Loads the JSON of the corresponding task. When the latest date was.
        Then changes task.num_days starting from the latest date.
        """
        pass

In [3]:
PAIRS_HIGH_CAP = [
    "BTC/USDT","ETH/USDT","ADA/USDT","BNB/USDT","XRP/USDT","DOT/USDT","DOGE/USDT","ZEC/USDT","NANO/USDT","NEO/USDT",
    "UNI/USDT","BCH/USDT","LINK/USDT","LTC/USDT","MATIC/USDT","XLM/USDT","SOL/USDT","ETC/USDT","VET/USDT","THETA/USDT",
    "EOS/USDT","TRX/USDT","FIL/USDT","XMR/USDT","AAVE/USDT","MKR/USDT","ATOM/USDT","ALGO/USDT","CAKE/USDT","KSM/USDT",
    "LUNA/USDT","BTT/USDT","AVAX/USDT","COMP/USDT","DASH/USDT","DCR/USDT","EGLD/USDT","WAVES/USDT","YFI/USDT","XEM/USDT",
    "CHZ/USDT","SUSHI/USDT","HOT/USDT","ZIL/USDT","SNX/USDT","MANA/USDT","ENJ/USDT","HNT/USDT","BAT/USDT","NEAR/USDT",
    "QTUM/USDT","GRT/USDT","ONE/USDT","ONT/USDT","BAKE/USDT","BNT/USDT","ZRX/USDT","FTM/USDT","OMG/USDT","CELO/USDT",
    "ICX/USDT","ANKR/USDT","RVN/USDT","CRV/USDT", "FTT/USDT", "TFUEL/USDT"
]
PAIRS_BLVT_DOWN = [
    "BTCDOWN/USDT", "BNBDOWN/USDT", "ETHDOWN/USDT", "AAVEDOWN/USDT", "XRPDOWN/USDT", "ADADOWN/USDT", "SUSHIDOWN/USDT",
    "DOTDOWN/USDT", "1INCHDOWN/USDT", "LINKDOWN/USDT", "UNIDOWN/USDT", "SXPDOWN/USDT", "EOSDOWN/USDT", "BCHDOWN/USDT",
    "YFIDOWN/USDT", "XLMDOWN/USDT", "FILDOWN/USDT", "TRXDOWN/USDT", "XTZDOWN/USDT", "LTCDOWN/USDT",
]
pairs = PAIRS_HIGH_CAP

In [4]:
# NOTE: Don't use 1 month (1M) timeframe as it will overwrite the 1 minute (1m) in Windows.
timeframes = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d"]
exchange = "binance"
num_days = 30
num_threads = 10

tasks = []

# Prepare the job
for tf in timeframes:
    for pair in pairs:
        new_task = Task(pair, tf, num_days)
        tasks.append(new_task)

downloader = Downloader(tasks, exchange, PATH_LOCAL_DATADIR)
downloader.start(num_threads)

Got 792 tasks.
Datadir: c:\CS\Python\crypto-collection\freqtrade\mount\data\binance
Starting threads
Thread #0 - Download ['TFUEL/USDT'] ['1d']. Tasks left: 791
Starting worker #0
Starting worker #1
Thread #1 - Download ['FTT/USDT'] ['1d']. Tasks left: 790
Starting worker #2
Thread #2 - Download ['CRV/USDT'] ['1d']. Tasks left: 789
Starting worker #3
Thread #3 - Download ['RVN/USDT'] ['1d']. Tasks left: 788
Starting worker #4
Thread #4 - Download ['ANKR/USDT'] ['1d']. Tasks left: 787
Starting worker #5
Thread #5 - Download ['ICX/USDT'] ['1d']. Tasks left: 786
Starting worker #6
Thread #6 - Download ['CELO/USDT'] ['1d']. Tasks left: 785
Starting worker #7
Thread #7 - Download ['OMG/USDT'] ['1d']. Tasks left: 784
Starting worker #8
Thread #8 - Download ['FTM/USDT'] ['1d']. Tasks left: 783
Starting worker #9
Thread #9 - Download ['ZRX/USDT'] ['1d']. Tasks left: 782


c:\CS\Python\crypto-collection\freqtrade\superft\freqtrade\data\history\jsondatahandler.py:65: FutureWarning: casting datetime64[ns, UTC] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  _data['date'] = _data['date'].astype(np.int64) // 1000 // 1000


Thread #8 - Download ['BNT/USDT'] ['1d']. Tasks left: 781
Thread #4 - Download ['BAKE/USDT'] ['1d']. Tasks left: 780
Thread #6 - Download ['ONT/USDT'] ['1d']. Tasks left: 779
Thread #2 - Download ['ONE/USDT'] ['1d']. Tasks left: 778
Thread #1 - Download ['GRT/USDT'] ['1d']. Tasks left: 777
Thread #0 - Download ['NEAR/USDT'] ['1d']. Tasks left: 775
Thread #7 - Download ['QTUM/USDT'] ['1d']. Tasks left: 776
Thread #3 - Download ['BAT/USDT'] ['1d']. Tasks left: 774
Thread #5 - Download ['HNT/USDT'] ['1d']. Tasks left: 773
Thread #9 - Download ['ENJ/USDT'] ['1d']. Tasks left: 772
Thread #8 - Download ['MANA/USDT'] ['1d']. Tasks left: 771
Thread #4 - Download ['SNX/USDT'] ['1d']. Tasks left: 770
Thread #0 - Download ['ZIL/USDT'] ['1d']. Tasks left: 769
Thread #7 - Download ['HOT/USDT'] ['1d']. Tasks left: 768
Thread #1 - Download ['SUSHI/USDT'] ['1d']. Tasks left: 767
Thread #3 - Download ['CHZ/USDT'] ['1d']. Tasks left: 766
Thread #9 - Download ['XEM/USDT'] ['1d']. Tasks left: 765
Thread #

In [ ]:
BINANCE = PATH_LOCAL_DATADIR / "binance"
datasets = {
    "binance_highcap": BINANCE.glob("*.json"),
    "binance_down": BINANCE.glob("*DOWN*.json"),
}

# Upload data to wandb
DATASET_NAME = "binance_highcap"

path_to_pairs = datasets[DATASET_NAME]

with wandb.init(project="ft-datasets") as run:
    artifact = wandb.Artifact(DATASET_NAME, type="dataset")
    [artifact.add_file(it) for it in path_to_pairs]
    run.log_artifact(artifact)